## Working directory set-up

In [ ]:
%%capture
_drive = False
_local = True
try: 
  from google.colab import drive
  drive.mount('/content/drive')
  _drive = True
  _local = False
except: 
  print('no google drive')

In [ ]:
cd drive/MyDrive/PhD/Thesis/Dissertation/Code

In [ ]:
!pwd

In [ ]:
!pip install tensorflow_addons
!pip install imblearn
!pip install openpyxl 
!pip install matplotlib

## Imports

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization, LSTM, Conv1D
from keras.models import Sequential
from dataclasses import dataclass
import tensorflow_datasets as tfds 
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from imblearn.over_sampling import SMOTE, RandomOverSampler
import os
from   datetime import date

from keras import callbacks
import tensorflow_addons as tfa

import pandas as pd
import numpy as np
from statistics import mean

from datetime import datetime
import time
import glob
import re
from pprint import pprint
import matplotlib.pyplot as plt

In [ ]:
@dataclass
class Config:
    # preprocessing params
    MAX_LEN = 256      
    VOCAB_SIZE = 30000  

    # model params
    EMBED_DIM = 128
    FF_DIM = 128  
    NUM_LAYERS = 3 

    LR = 0.001   
    DROPOUT = 0.1 
    BATCH_SIZE = 32     

    # transformer 
    NUM_HEAD = 8 

    # parallelism
    BUFFER_SIZE = tf.data.experimental.AUTOTUNE

    CODE_VERSION = 'Not defined'

    def save(self, fname):
      dict1 = {'MAX_LEN' : self.MAX_LEN, 
               'VOCAB_SIZE' : self.VOCAB_SIZE,

               'EMBED_DIM' : self.EMBED_DIM,
               'BATCH_SIZE' : self.BATCH_SIZE,

               'LR' : self.LR,
               'DROPOUT' : self.DROPOUT,
               'FF_DIM' : self.FF_DIM,
               'NUM_LAYERS' : self.NUM_LAYERS,

               'CODE_VERSION' : self.CODE_VERSION,
               
               }
      file1 = open(fname, "w") 
      str1 = repr(dict1)
      file1.write("config = " + str1 + "\n")
      file1.close()

config = Config()

## Data Preprocessing

### Tokenization

In [ ]:
def get_vectorize_layer( vocab_size, max_seq, vec_log='tv_layer.pkl'):

    try: 
      from_disk = pickle.load(open(vec_log, "rb"))
      vectorize_layer = TextVectorization.from_config(from_disk['config'])
      vectorize_layer.set_weights(from_disk['weights'])
      print('vectorize layer already existed')

    except:
      print('vectorize layer not found')

      vectorize_layer = TextVectorization(
          max_tokens=vocab_size,
          output_mode="int",
          standardize=None,
          output_sequence_length=max_seq,
      )
      text_ds = tf.data.Dataset.from_tensor_slices(texts).prefetch(config.BUFFER_SIZE)
      vectorize_layer.adapt(text_ds)

      # Insert mask token in vocabulary
      vocab = vectorize_layer.get_vocabulary()
      vocab = vocab[2 : vocab_size ]
      vectorize_layer.set_vocabulary(vocab)


    return vectorize_layer

In [ ]:
def encode(texts):
    texts = tf.expand_dims(texts, -1)  
    encoded_texts = tf.squeeze(vectorize_layer(texts))
    return encoded_texts

### Data balancing

In [ ]:
def data_balancing(x, y):

  ROS = SMOTE(sampling_strategy=1, k_neighbors =3)
  X_bal, y_bal = ROS.fit_resample(x, y)

  return X_bal, y_bal

## Load labeled datasets

In [ ]:
def split_data(data, training_ratio, _valid=False):

  #train_size=0.8

  X = data.drop(columns = ['label']).copy()
  y = data['label']

  X_train, X_rem, y_train, y_rem = train_test_split(X,y,stratify = y, train_size=training_ratio)

  if  _valid == False:
    return X_train, X_rem, y_train, y_rem
  
  else:
    X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)
    return  X_train, X_valid, X_test, y_train, y_valid, y_test


In [ ]:
def load_labeled_dataset(fname, training_ratio=0.7):
  dataset = pd.read_csv (fname, index_col =None, header =0)

  all_data = dataset[['Code', 'Smelly']]
  all_data = all_data.rename(columns={"Code": "code", "Smelly": "label"})

  train_df = pd.DataFrame()
  test_df = pd.DataFrame()
  X_train,  X_test, y_train, y_test= split_data(all_data, training_ratio= training_ratio)
  train_df = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
  test_df = pd.concat([X_test, y_test], axis=1).reset_index(drop=True)


  return train_df, test_df

In [ ]:
def get_finetuning_dataset(train_df, test_df):
# Prepare datasets for training

  x_train = encode(train_df.code.values)
  y_train = train_df.label.values

  # for testing
  x_test = encode(test_df.code.values)
  y_test = test_df.label.values

  return x_train, y_train, x_test, y_test #train_ds, test_ds

In [ ]:
def data_preprocessing(x_train, y_train, x_test, y_test, balance = False):
  if balance:
    #print(Counter(y_train))
    x_train, y_train = data_balancing(x_train, y_train)
    print(Counter(y_train))
    x_test, y_test = data_balancing( x_test, y_test)
    print(Counter(y_test))

  return x_train, y_train, x_test, y_test


## Fine-tuning and Feature-based

In [ ]:
def create_classifier_model(pretrained_model):
    inputs = layers.Input((config.MAX_LEN,), dtype=tf.int64)
    sequence_output = pretrained_model(inputs)
    pooled_output = layers.GlobalMaxPooling1D()(sequence_output)
    outputs = layers.Dense(1, activation="sigmoid")(pooled_output)
    classifer_model = keras.Model(inputs, outputs, name="classification")
    optimizer = keras.optimizers.Adam()
    classifer_model.compile(
        optimizer=optimizer, loss=loss, metrics=metrics
    )
    return classifer_model

In [ ]:
def fine_tune(pretrained_model, x_train, y_train, x_test, y_test):
    print('================== Start training ==================\n')
    # Freeze it
    pretrained_model.trainable = False
    classifer_model = create_classifier_model(pretrained_model)
    #classifer_model.summary()

    # Train the classifier with frozen  stage (feature-based)
    start_time = time.time()
    classifer_model.fit(x_train, y_train, epochs=epochs, validation_data=[x_test, y_test ], verbose=0)
    train_time_1 = round((time.time() - start_time))
    history_1 = classifer_model.evaluate(x_test, y_test)

    # Unfreeze the model for fine-tuning
    pretrained_model.trainable = True
    classifer_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    start_time = time.time()
    classifer_model.fit(x_train, y_train, epochs=epochs, validation_data=[x_test, y_test], verbose=0)
    train_time_2 = round((time.time() - start_time))
    
    start_time = time.time()
    history_2 = classifer_model.evaluate(x_test, y_test)
    EVAL_TIME = round((time.time() - start_time))
    print('EVAL_TIME = ', EVAL_TIME)

    return history_1, history_2, train_time_1, train_time_2, classifer_model, #EVAL_TIME

In [ ]:
def fine_tune_feature_based(pretrained_model, x_train, y_train, x_test, y_test):
    print('================== Start training ==================\n')
    # Freeze it
    pretrained_model.trainable = False
    classifer_model = create_classifier_model(pretrained_model)
    #classifer_model.summary()

    # Train the classifier with frozen  stage
    start_time = time.time()
    classifer_model.fit(x_train, y_train, epochs=epochs, validation_data=[x_test, y_test ], verbose=0)
    train_time_1 = round((time.time() - start_time))
    start_time = time.time()
    history_1 = classifer_model.evaluate(x_test, y_test)
    eval_time_1 = round((time.time() - start_time))


    return history_1,  eval_time_1, classifer_model

##  Helper functions

In [ ]:
def getToday ():
  return str(date.today())

def getTime ():
  t = time.localtime()
  current_time = time.strftime('%H-%M-%S', t)
  return current_time

def Save_Excel (result, fname, mode='w'):
  
  if not os.path.isfile(fname) : result.to_csv (fname, header='column_names')
  else : 
    result.to_csv(fname,  mode='a', header=False)


In [ ]:
def mcc(tp,fp,tn,fn):
    try:   
        return (tp*tn - fp*fn)/((tp+fp)*(tp+fn)*(fn+tn)*(fp+tn))**0.5
    except:
        return 0

In [ ]:
def get_result(history, time,  proj, m_name, eval_proj=None, n_samples =-1):
    
    if (n_samples == -1) and (eval_proj==None):
        results = pd.DataFrame  (columns = ['Dataset', 'Model', 'Time (Sec)', 'Accuracy', 'Precision', 'Recall',
                                            'F1-score', 'AUC','MCC','TP', 'TN', 'FP',  'FN'])
        pres = np.round(history[2] * 100,2)
        rec = np.round(history[3] * 100,2)

        results = results.append({'Dataset'    : proj[0:-4],
                            'Model' : m_name,
                            'Time (Sec)'       : time,
                            'Accuracy'   : np.round(history[1] * 100,2),
                            'Precision'  : pres,
                            'Recall'     : rec, 
                            'F1-score'   : np.round( 2* ((pres*rec)/(pres+rec)),2),
                            'AUC'        : np.round(history[4] * 100,2),
                            'MCC'        : np.round(mcc(history[5],history[7],history[6],history[8]) * 100,2),
                            'TP'         : history[5],
                            'TN'         : history[6],
                            'FP'         : history[7],
                            'FN'         : history[8],
                            },
                            ignore_index = True)
    
    elif n_samples != -1:
        results = pd.DataFrame  (columns = ['Dataset', 'Model', 'Size', 'Time (Sec)', 'Accuracy', 'Precision', 'Recall',
                                            'F1-score', 'AUC','MCC','TP', 'TN', 'FP',  'FN'])
        pres = np.round(history[2] * 100,2)
        rec = np.round(history[3] * 100,2)

        results = results.append({'Dataset'    : proj[0:-4],
                            'Model' : m_name,
                            'Size'  : n_samples,
                            'Time (Sec)'       : time,
                            'Accuracy'   : np.round(history[1] * 100,2),
                            'Precision'  : pres,
                            'Recall'     : rec, 
                            'F1-score'   : np.round( 2* ((pres*rec)/(pres+rec)),2),
                            'AUC'        : np.round(history[4] * 100,2),
                            'MCC'        : np.round(mcc(history[5],history[7],history[6],history[8]) * 100,2),
                            'TP'         : history[5],
                            'TN'         : history[6],
                            'FP'         : history[7],
                            'FN'         : history[8],
                            },
                            ignore_index = True)
    elif eval_proj != None:
        results = pd.DataFrame  (columns = ['Training dataset', 'Evaluation dataset', 'Model', 'Time (Sec)', 'Accuracy', 'Precision', 'Recall',
                                            'F1-score', 'AUC','MCC','TP', 'TN', 'FP',  'FN'])
        pres = np.round(history[2] * 100,2)
        rec = np.round(history[3] * 100,2)

        results = results.append({'Training dataset'    : proj[0:-4],
                            'Evaluation dataset' : eval_proj[0:-4],
                            'Model'  : m_name,
                            'Time (Sec)'       : time,
                            'Accuracy'   : np.round(history[1] * 100,2),
                            'Precision'  : pres,
                            'Recall'     : rec, 
                            'F1-score'   : np.round( 2* ((pres*rec)/(pres+rec)),2),
                            'AUC'        : np.round(history[4] * 100,2),
                            'MCC'        : np.round(mcc(history[5],history[7],history[6],history[8]) * 100,2),
                            'TP'         : history[5],
                            'TN'         : history[6],
                            'FP'         : history[7],
                            'FN'         : history[8],
                            },
                            ignore_index = True)
        
        
    
    return results
    

## _ Main

In [ ]:
vec_layer_logs = 'vec_layer_logs/'+''+'tv_layer.pkl'

# # Get vectorize layer
vectorize_layer = get_vectorize_layer(
    #train_df.code.values.tolist(),
    config.VOCAB_SIZE,
    config.MAX_LEN, 
    vec_log= vec_layer_logs,
)

In [ ]:
# for loops cs>>models>>proj
not_completed = []

CODE_SMELLS = ['Data class', 'God class', 'Feature envy', 'Long method']

DATA_PATH = 'Data/Fine-tuning/Code smells/'
SAVE_PATH = 'Results/' + getToday() + '_' + getTime()

# Set training parameters 
optimizer = keras.optimizers.Adam()
loss="binary_crossentropy"
metrics=['accuracy', tf.metrics.Precision(), tf.metrics.Recall(),  tf.metrics.AUC(),
         tf.keras.metrics.TruePositives(), tf.keras.metrics.TrueNegatives(), 
         tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives()]
epochs=5


In [ ]:
MODEL_NAME = {'ANN':'logs/hparam_tuning/DL 16-11-2022/run-18/dl_model.h5',
              'CNN':'logs/hparam_tuning/CNN 06-12-2022/run-20/cnn_model.h5', 
              'LSTM': 'logs/hparam_tuning/LSTM 21-11-2022/run-19/lstm_model.h5',
              'Transformer':'logs/hparam_tuning/Transformer 15-12-2022/run-6/transformer_model.h5',
              }

#CODE_SMELLS = ['Long method']

m_name = 'Transformer' #item[0]
m_best = MODEL_NAME[m_name] #item[1]
print(m_name, m_best)

# Load pretrained model
model_path = m_best #"transformer_model.h5"
loaded_model = keras.models.load_model(model_path, custom_objects={"DL_Model": 'DLModel'})
pretrained_model = tf.keras.Model(loaded_model.input, loaded_model.get_layer(index=len(loaded_model.layers)-1).output)

In [ ]:

for code_smell in CODE_SMELLS:
    
    PROJECTS = os.listdir(DATA_PATH + code_smell +'/') 

    for proj in PROJECTS:

        fname = DATA_PATH + code_smell +'/' + proj
        print(fname)

        try:

            train_df, test_df= load_labeled_dataset(fname)
            x_train, y_train, x_test, y_test =  get_finetuning_dataset(train_df, test_df)
            x_train, y_train, x_test, y_test =  data_preprocessing(x_train, y_train, x_test, y_test, balance=True)


        except:
            print('Data has less than 6 positive labels: ', fname)
            continue
        

        try:
              # fine tuning 
                
              history_1, history_2, train_time_1, train_time_2, _ = fine_tune(pretrained_model, x_train, y_train, x_test, y_test)

              print(history_2)

              # Save results

              fine_tuning_results =  get_result(history_2, train_time_2,  proj, m_name)
              feature_based_results = get_result(history_1, train_time_1,  proj, m_name)

              Save_Excel (fine_tuning_results,  SAVE_PATH + '_' + code_smell+ '_FineTuningResults_SMOTE_'+m_name+'.csv')
              Save_Excel (feature_based_results,  SAVE_PATH +'_' + code_smell+ '_FeatureBasedResults_SMOTE_'+m_name+'.csv')

        except Exception as e: 
              print(e)
              not_completed.append((code_smell, proj))

print(not_completed)

## Evaluation

### Unseen projects

In [ ]:
def cross_proj_evaluation(model, m_name, file_path, train_proj, eval_projects):
  results = pd.DataFrame()
  for proj in eval_projects:
    fname = file_path+ proj
    print('-------------- Evaluation on ---------------\n', fname)

    try:
        train_df, _ = load_labeled_dataset(fname, training_ratio=0.99)
        x_train, y_train, _, _ =  get_finetuning_dataset(train_df, train_df)
        _, _, x_test, y_test =  data_preprocessing(x_train, y_train, x_train, y_train, balance=True)

    except  Exception as e: 
        print(e)
        print('Data has less than 6 positive labels: ', fname)
        continue
    start_time = time.time()
    history = model.evaluate(x_test, y_test)
    train_time = round((time.time() - start_time))
    
    fine_tuning_results =  get_result(history, train_time,  train_proj , m_name, eval_proj=proj)
    results = results.append(fine_tuning_results)


  return results

In [ ]:
def get_evaluation_projects(code_smell):

  eval_projects = pd.read_csv ('cross-projects.csv', index_col =None, header =0)
  eval_projects = pd.unique(eval_projects[code_smell])
  eval_projects =  [x+'.csv' for x in eval_projects if str(x) != 'nan']

  return eval_projects

In [ ]:
for code_smell in CODE_SMELLS:
    
    PROJECTS = get_evaluation_projects(code_smell)

    for proj in PROJECTS:

        fname = DATA_PATH + code_smell +'/' + proj 
        print(fname)

        try:

            train_df, test_df= load_labeled_dataset(fname)
            x_train, y_train, x_test, y_test =  get_finetuning_dataset(train_df, test_df)
            x_train, y_train, x_test, y_test =  data_preprocessing(x_train, y_train, x_test, y_test, balance=True)

        except:
            print('Data has less than 6 positive labels: ', fname)
            continue

        try:
          # fine tuning 

          #history_1, history_2, train_time_1, train_time_2, model = fine_tune(pretrained_model, x_train, y_train, x_test, y_test)
          history_1, eval_time_1, model = fine_tune_feature_based(pretrained_model, x_train, y_train, x_test, y_test)
          #print(history_2)

          # Save results

          #fine_tuning_results =  get_result(history_2, 0,  proj, m_name, eval_proj = proj)
          #cross_proj_results =  cross_proj_evaluation(model, m_name, DATA_PATH + code_smell +'/' , proj , sorted(set(PROJECTS)-set([proj])))
          #fine_tuning_results = fine_tuning_results.append(cross_proj_results)      
         
          feature_based_results = get_result(history_1, eval_time_1,  proj, m_name, eval_proj = proj)
          cross_proj_results =  cross_proj_evaluation(model, m_name, DATA_PATH + code_smell +'/' , proj , sorted(set(PROJECTS)-set([proj])))
          feature_based_results = feature_based_results.append(cross_proj_results)  
          
          #Save_Excel (fine_tuning_results,  SAVE_PATH + '_' + code_smell+ '_FineTuningResults_CrossProj_'+m_name+'.csv')
          Save_Excel (feature_based_results,  SAVE_PATH +'_' + code_smell+ '_FeatureBasedResults_CrossProj_'+m_name+'.csv')

        except Exception as e: 
          print(e)
          not_completed.append((code_smell, proj))

print(not_completed)

### Dataset size

In [ ]:
def load_labeled_dataset_with_size(fname, n_samples):
  dataset = pd.read_csv (fname, index_col =None, header =0)

  all_data = dataset[['Code', 'Smelly']]
  all_data = all_data.rename(columns={"Code": "code", "Smelly": "label"})
  
  all_data = all_data.sample(n=n_samples, random_state=1).reset_index(drop=True)#.sort_values('label', ascending =False)#

  X_train,  X_test, y_train, y_test= split_data(all_data, training_ratio= 0.7)

  train_df = pd.DataFrame()
  test_df = pd.DataFrame()

  train_df = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
  test_df = pd.concat([X_test, y_test], axis=1).reset_index(drop=True)


  return train_df, test_df

In [ ]:
# define dataset sizes to evaluate
sizes = [100, 500, 1000, 1500]
score_sets, means = list(), list()
not_completed = []

for code_smell in CODE_SMELLS:
    
    PROJECTS = os.listdir(DATA_PATH + code_smell +'/') 
    #PROJECTS = ['accumulo.csv', 'airavata.csv',]

    for proj in PROJECTS:

        fname = DATA_PATH + code_smell +'/' + proj
        print(fname)
        for n_samples in sizes: 
          try:

              train_df, test_df = load_labeled_dataset_with_size(fname, n_samples)
              x_train, y_train, x_test, y_test =  get_finetuning_dataset(train_df, test_df)
              x_train, y_train, x_test, y_test =  data_preprocessing(x_train, y_train, x_test, y_test, balance=True)

          except:
              print('Data has less than 6 positive labels: ', fname)
              break#continue

          try:
            x=1
            # fine tuning 
            history_1, history_2, train_time_1, train_time_2, _ = fine_tune(pretrained_model, x_train, y_train, x_test, y_test)
            #print(history_2)
            #history_1, train_time_1, model = baseline_model( x_train, y_train, x_test, y_test)

            # Save results

            dataset_size_tuning_results =  get_result(history_2, train_time_2,  proj, m_name, None, n_samples)
            dataset_size_feature_results = get_result(history_1, train_time_1,  proj, m_name, None, n_samples)

            Save_Excel (dataset_size_tuning_results,  SAVE_PATH + '_' + code_smell+ '_FineTuningResults_Size_'+m_name+'.csv')
            Save_Excel (dataset_size_feature_results,  SAVE_PATH +'_' + code_smell+ '_FeatureBasedResults_Size_'+m_name+'.csv')

          except:
            x=6
            not_completed.append((code_smell, proj))
          
          print('Size=%d: %s' % (n_samples,fname))

print(not_completed)

### Baseline

In [ ]:
def baseline_model( x_train, y_train, x_test, y_test):

  regression_layer = keras.layers.Dense(1, activation="sigmoid")
  inputs = keras.Input(shape=(config.MAX_LEN))
  outputs = regression_layer(inputs)
  baseline_model = keras.Model(inputs, outputs)
  baseline_model.compile(loss=loss, metrics=metrics)

  start_time = time.time()
  baseline_model.fit(x_train, y_train, epochs=epochs, validation_data=[x_test, y_test ], verbose=0)
  train_time_1 = round((time.time() - start_time))
  history_1 = baseline_model.evaluate(x_test, y_test)

  return history_1,train_time_1, baseline_model

In [ ]:
m_name = 'Baseline'
not_completed=[]
for code_smell in CODE_SMELLS:
    
    #PROJECTS = os.listdir(DATA_PATH + code_smell +'/') 
    #PROJECTS = ['activemq.csv', 'airavata.csv',]
    PROJECTS = get_evaluation_projects(code_smell)

    for proj in PROJECTS:

        fname = DATA_PATH + code_smell +'/' + proj
        print(fname)

        try:

            train_df, test_df= load_labeled_dataset(fname)
            x_train, y_train, x_test, y_test =  get_finetuning_dataset(train_df, test_df)
            x_train, y_train, x_test, y_test =  data_preprocessing(x_train, y_train, x_test, y_test, balance=True)


        except:
            print('Data has less than 6 positive labels: ', fname)
            continue


        try:
          # fine tuning 
          history_1, train_time_1, model = baseline_model( x_train, y_train, x_test, y_test)

          # Save results
          baseline_results = get_result(history_1, train_time_1,  proj, m_name)
          Save_Excel (baseline_results,  SAVE_PATH +'_' + code_smell+ '_BaselineResults.csv')
        
          
          #cross projects evaluation
          baseline_results = get_result(history_1, train_time_1,  proj, m_name, eval_proj = proj)
          cross_proj_results =  cross_proj_evaluation(model, m_name, DATA_PATH + code_smell +'/' , proj , sorted(set(PROJECTS)-set([proj])))
          baseline_results = baseline_results.append(cross_proj_results)  
          Save_Excel (baseline_results,  SAVE_PATH +'_' + code_smell+ '_BaselineResults_CrossProj_'+m_name+'.csv')

        except Exception as e:
          print(e)
          not_completed.append((code_smell, proj))

print(not_completed)